<a href="https://colab.research.google.com/github/youngscientist2k3/codemagic-sample-projects/blob/main/Technical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X=pd.read_excel('/content/5 Shares.xlsx', sheet_name=1, header=0)
Y=pd.read_excel('/content/5 Shares.xlsx', sheet_name=2, header=0)
Z=pd.read_excel('/content/5 Shares.xlsx', sheet_name=3, header=0)
A=pd.read_excel('/content/5 Shares.xlsx', sheet_name=4, header=0)

In [ ]:
all_data=pd.concat([X,Y,Z,A])

In [ ]:
all_data.columns

Index(['symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [ ]:
#importing variables
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader as pdr
import seaborn as sns
import matplotlib.pyplot as plt


#Creating Return column
all_data['return'] = all_data.groupby('symbol')['Close'].pct_change()

In [ ]:
all_data['SMA_5'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA_15'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_ratio'] = all_data['SMA_15'] / all_data['SMA_5']

In [ ]:
all_data['SMA5_Volume'] = all_data.groupby('symbol')['Volume'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA15_Volume'] = all_data.groupby('symbol')['Volume'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_Volume_Ratio'] = all_data['SMA5_Volume']/all_data['SMA15_Volume']

In [ ]:
def Wilder(data, periods):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].mean() #Simple Moving Average
    for i in range(start+periods,len(data)):
        Wilder[i] = (Wilder[i-1]*(periods-1) + data[i])/periods #Wilder Smoothing
    return(Wilder)

In [ ]:
all_data['prev_close'] = all_data.groupby('symbol')['Close'].shift(1)
all_data['TR'] = np.maximum((all_data['High'] - all_data['Low']), 
                     np.maximum(abs(all_data['High'] - all_data['prev_close']), 
                     abs(all_data['prev_close'] - all_data['Low'])))
for i in all_data['symbol'].unique():
    TR_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'ATR_5'] = Wilder(TR_data['TR'], 5)
    all_data.loc[all_data.symbol==i,'ATR_15'] = Wilder(TR_data['TR'], 15)

all_data['ATR_Ratio'] = all_data['ATR_5'] / all_data['ATR_15']

In [ ]:
all_data['prev_high'] = all_data.groupby('symbol')['High'].shift(1)
all_data['prev_low'] = all_data.groupby('symbol')['Low'].shift(1)

all_data['+DM'] = np.where(~np.isnan(all_data.prev_high),
                           np.where((all_data['High'] > all_data['prev_high']) & 
         (((all_data['High'] - all_data['prev_high']) > (all_data['prev_low'] - all_data['Low']))), 
                                                                  all_data['High'] - all_data['prev_high'], 
                                                                  0),np.nan)

all_data['-DM'] = np.where(~np.isnan(all_data.prev_low),
                           np.where((all_data['prev_low'] > all_data['Low']) & 
         (((all_data['prev_low'] - all_data['Low']) > (all_data['High'] - all_data['prev_high']))), 
                                    all_data['prev_low'] - all_data['Low'], 
                                    0),np.nan)

for i in all_data['symbol'].unique():
    ADX_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'+DM_5'] = Wilder(ADX_data['+DM'], 5)
    all_data.loc[all_data.symbol==i,'-DM_5'] = Wilder(ADX_data['-DM'], 5)
    all_data.loc[all_data.symbol==i,'+DM_15'] = Wilder(ADX_data['+DM'], 15)
    all_data.loc[all_data.symbol==i,'-DM_15'] = Wilder(ADX_data['-DM'], 15)

all_data['+DI_5'] = (all_data['+DM_5']/all_data['ATR_5'])*100
all_data['-DI_5'] = (all_data['-DM_5']/all_data['ATR_5'])*100
all_data['+DI_15'] = (all_data['+DM_15']/all_data['ATR_15'])*100
all_data['-DI_15'] = (all_data['-DM_15']/all_data['ATR_15'])*100

all_data['DX_5'] = (np.round(abs(all_data['+DI_5'] - all_data['-DI_5'])/(all_data['+DI_5'] + all_data['-DI_5']) * 100))

all_data['DX_15'] = (np.round(abs(all_data['+DI_15'] - all_data['-DI_15'])/(all_data['+DI_15'] + all_data['-DI_15']) * 100))

for i in all_data['symbol'].unique():
    ADX_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'ADX_5'] = Wilder(ADX_data['DX_5'], 5)
    all_data.loc[all_data.symbol==i,'ADX_15'] = Wilder(ADX_data['DX_15'], 15)

In [ ]:
all_data['Lowest_5D'] = all_data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = 5).min())
all_data['High_5D'] = all_data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = 5).max())
all_data['Lowest_15D'] = all_data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = 15).min())
all_data['High_15D'] = all_data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = 15).max())

all_data['Stochastic_5'] = ((all_data['Close'] - all_data['Lowest_5D'])/(all_data['High_5D'] - all_data['Lowest_5D']))*100
all_data['Stochastic_15'] = ((all_data['Close'] - all_data['Lowest_15D'])/(all_data['High_15D'] - all_data['Lowest_15D']))*100

all_data['Stochastic_%D_5'] = all_data['Stochastic_5'].rolling(window = 5).mean()
all_data['Stochastic_%D_15'] = all_data['Stochastic_5'].rolling(window = 15).mean()

all_data['Stochastic_Ratio'] = all_data['Stochastic_%D_5']/all_data['Stochastic_%D_15']

In [ ]:
all_data['Diff'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.diff())
all_data['Up'] = all_data['Diff']
all_data.loc[(all_data['Up']<0), 'Up'] = 0

all_data['Down'] = all_data['Diff']
all_data.loc[(all_data['Down']>0), 'Down'] = 0 
all_data['Down'] = abs(all_data['Down'])

all_data['avg_5up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=5).mean())
all_data['avg_5down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=5).mean())

all_data['avg_15up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=15).mean())
all_data['avg_15down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=15).mean())

all_data['RS_5'] = all_data['avg_5up'] / all_data['avg_5down']
all_data['RS_15'] = all_data['avg_15up'] / all_data['avg_15down']

all_data['RSI_5'] = 100 - (100/(1+all_data['RS_5']))
all_data['RSI_15'] = 100 - (100/(1+all_data['RS_15']))

all_data['RSI_ratio'] = all_data['RSI_5']/all_data['RSI_15']

In [ ]:
all_data['5Ewm'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=5, adjust=False).mean())
all_data['15Ewm'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=15, adjust=False).mean())
all_data['MACD'] = all_data['15Ewm'] - all_data['5Ewm']

In [ ]:
all_data['15MA'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window=15).mean())
all_data['SD'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window=15).std())
all_data['upperband'] = all_data['15MA'] + 2*all_data['SD']
all_data['lowerband'] = all_data['15MA'] - 2*all_data['SD']

In [ ]:
all_data['RC'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = 15)) 

In [ ]:
all_data.columns

Index(['symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'return',
       'SMA_5', 'SMA_15', 'SMA_ratio', 'SMA5_Volume', 'SMA15_Volume',
       'SMA_Volume_Ratio', 'prev_close', 'TR', 'ATR_5', 'ATR_15', 'ATR_Ratio',
       'prev_high', 'prev_low', '+DM', '-DM', '+DM_5', '-DM_5', '+DM_15',
       '-DM_15', '+DI_5', '-DI_5', '+DI_15', '-DI_15', 'DX_5', 'DX_15',
       'ADX_5', 'ADX_15', 'Lowest_5D', 'High_5D', 'Lowest_15D', 'High_15D',
       'Stochastic_5', 'Stochastic_15', 'Stochastic_%D_5', 'Stochastic_%D_15',
       'Stochastic_Ratio', 'Diff', 'Up', 'Down', 'avg_5up', 'avg_5down',
       'avg_15up', 'avg_15down', 'RS_5', 'RS_15', 'RSI_5', 'RSI_15',
       'RSI_ratio', '5Ewm', '15Ewm', 'MACD', '15MA', 'SD', 'upperband',
       'lowerband', 'RC'],
      dtype='object')

In [ ]:
all_data=all_data.rename(columns={'symbol':'Symbol','return':'Return', '+DM':'plus DM', '-DM':'minus DM', '+DM_5': 'plus DM_5', '-DM_5':'minus DM_5', '+DM_15':'plus DM_15',
       '-DM_15': ' minus DM_15', '+DI_5':'plus DI_5', '-DI_5':'minus DI_5', '+DI_15':'plus DI_15', '-DI_15':'minus DI_15'})

In [ ]:
all_data.dropna()

,Symbol,Date,Open,High,Low,Close,Volume,Return,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio,prev_close,TR,ATR_5,ATR_15,ATR_Ratio,prev_high,prev_low,plus DM,minus DM,plus DM_5,minus DM_5,plus DM_15,minus DM_15,plus DI_5,minus DI_5,plus DI_15,minus DI_15,DX_5,DX_15,ADX_5,ADX_15,Lowest_5D,High_5D,Lowest_15D,High_15D,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,Diff,Up,Down,avg_5up,avg_5down,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD,15MA,SD,upperband,lowerband,RC
29,SRF,1999-02-15 15:29:59.712,1.80,1.98,1.80,1.95,6800.0,0.083333,2.044,2.234000,1.092955,22990.0,11590.000000,1.983607,1.80,0.18,0.282575,0.252131,1.120748,2.10,1.77,0.00,0.00,0.032178,0.084839,0.058952,0.063305,11.387366,30.023454,23.381448,25.107880,45.0,4.0,40.548306,19.066667,1.77,2.39,1.77,3.12,29.032258,13.333333,14.963859,37.089506,0.403453,0.15,0.15,0.00,0.060,0.140,0.073333,0.076667,0.428571,0.956522,30.000000,48.888889,0.613636,2.030716,2.177635,0.146919,2.234000,0.276245,2.786490,1.681510,-0.025000
30,SRF,1999-02-16 15:29:59.712,1.83,1.90,1.58,1.60,81100.0,-0.179487,1.924,2.203333,1.145184,38650.0,16856.666667,2.292861,1.95,0.37,0.300060,0.259989,1.154127,1.98,1.80,0.00,0.22,0.025742,0.111871,0.055022,0.073751,8.579045,37.282883,21.163111,28.367018,63.0,15.0,45.038645,18.795556,1.58,2.39,1.58,3.12,2.469136,1.298701,15.457686,33.991704,0.454749,-0.35,0.00,0.35,0.060,0.180,0.069333,0.100000,0.333333,0.693333,25.000000,40.944882,0.610577,1.887144,2.105430,0.218286,2.203333,0.319143,2.841620,1.565047,-0.223301
31,SRF,1999-02-17 15:29:59.712,1.83,1.97,1.65,1.94,29500.0,0.212500,1.842,2.192667,1.190373,44150.0,18573.333333,2.377064,1.60,0.37,0.314048,0.267323,1.174790,1.90,1.58,0.07,0.00,0.034594,0.089497,0.056020,0.068834,11.015458,28.497815,20.956033,25.749509,44.0,10.0,44.830916,18.209185,1.58,2.39,1.58,3.12,44.444444,23.376623,17.679908,32.089802,0.550951,0.34,0.34,0.00,0.098,0.180,0.089333,0.100000,0.544444,0.893333,35.251799,47.183099,0.747128,1.904763,2.084752,0.179989,2.192667,0.325455,2.843577,1.541757,-0.076190
32,SRF,1999-02-18 15:29:59.712,1.94,1.99,1.50,1.71,59300.0,-0.118557,1.800,2.170667,1.205926,45240.0,22430.000000,2.016942,1.94,0.49,0.349238,0.282168,1.237697,1.97,1.65,0.00,0.15,0.027675,0.101597,0.052286,0.074245,7.924404,29.091146,18.529946,26.312470,57.0,17.0,47.264733,18.128573,1.50,2.10,1.50,3.12,35.000000,12.962963,22.929908,30.639351,0.748381,-0.23,0.00,0.23,0.098,0.140,0.089333,0.111333,0.700000,0.802395,41.176471,44.518272,0.924934,1.839842,2.037908,0.198066,2.170667,0.346955,2.864578,1.476756,-0.161765
33,SRF,1999-02-19 15:29:59.712,1.83,1.90,1.60,1.80,80850.0,0.052632,1.800,2.157333,1.198519,51510.0,27620.000000,1.864953,1.71,0.30,0.339391,0.283357,1.197751,1.99,1.50,0.00,0.00,0.022140,0.081278,0.048800,0.069296,6.523469,23.948199,17.222058,24.455272,57.0,17.0,49.211786,18.053335,1.50,1.99,1.50,3.12,61.224490,18.518519,34.434066,33.851419,1.017212,0.09,0.09,0.00,0.116,0.116,0.095333,0.108667,1.000000,0.877301,50.000000,46.732026,1.069930,1.826561,2.008169,0.181608,2.157333,0.357660,2.872654,1.442013,-0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,UTI,2021-11-19 16:00:00.288,7.81,8.56,7.76,8.50,550275.0,0.096774,7.572,7.278667,0.961261,201863.8,103093.200000,1.958071,7.75,0.81,0.577753,0.353306,1.635276,8.10,7.54,0.46,0.00,0.268255,0.055279,0.138578,0.042634,46.430757,9.567882,39.223262,12.067093,66.0,53.0,50.812177,22.168881,6.90,8.56,6.82,8.56,96.385542,96.551724,48.391144,65.911724,0.734181,0.75,0.75,0.00,0.330,0.154,0.164667,0.067333,2.142857,2.445545,68.181818,70.977011,0.960618,7.747625,7.359133,-0.388492,7.278667,0.412949,8.104564,6.452769,0.207386
4515,UTI,2021-11-22 16:00:00.288,8.50,9.04,8.00,9.04,518145.0,0.063529,7.914,7.41

In [ ]:
all_data.to_csv('alldata.csv')

In [ ]:
all_data['Bollinger'] = (all_data['Close']-all_data['lowerband'])/(all_data['upperband']- all_data['lowerband'])

In [ ]:
all_data=all_data.dropna()

In [ ]:
all_data.describe()

,Open,High,Low,Close,Volume,Return,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio,prev_close,TR,ATR_5,ATR_15,ATR_Ratio,prev_high,prev_low,plus DM,minus DM,plus DM_5,minus DM_5,plus DM_15,minus DM_15,plus DI_5,minus DI_5,plus DI_15,minus DI_15,DX_5,DX_15,ADX_5,ADX_15,Lowest_5D,High_5D,Lowest_15D,High_15D,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,Diff,Up,Down,avg_5up,avg_5down,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD,15MA,SD,upperband,lowerband,RC,Bollinger
count,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000
mean,850.173593,860.338065,838.954900,849.496132,9.989041e+05,0.001251,848.424588,845.646043,0.997781,9.982127e+05,9.975058e+05,1.003990,848.977886,22.584624,22.499188,22.332951,0.993399,859.798581,838.439363,5.225818,4.884045,5.216038,4.850803,5.175737,4.816019,22.429155,19.984040,22.786596,20.532666,40.633399,25.088900,40.641643,25.110702,822.075140,874.502594,797.509768,892.638980,51.004175,53.218730,51.007978,51.009553,1.003095,0.518246,5.837438,5.319192,5.835016,5.290702,5.810301,5.259532,inf,1.510437,51.882350,52.619702,1.002663,848.414628,845.655537,-2.759091,845.646043,22.565460,890.776963,800.515123,0.016320,0.537381
std,1812.590602,1831.394861,1791.165492,1810.209439,2.410711e+06,0.039498,1808.664057,1804.772787,0.040329,2.200063e+06,2.125259e+06,0.352309,1809.662649,58.447335,47.939390,45.475700,0.157639,1830.793660,1790.617342,30.237842,30.164457,13.753803,14.373002,11.056135,11.503320,12.767377,12.133773,8.439246,8.287301,25.850335,17.748107,15.537752,10.307473,1760.790514,1858.615729,1716.786605,1893.622569,28.524019,28.776343,20.959008,13.389602,0.355480,43.230312,30.566040,29.537656,17.272888,16.986920,13.514274,13.667435,NaN,1.543278,27.490902,16.667758,0.602604,1808.500314,1804.416229,39.116880,1804.772787,54.488836,1893.009702,1718.917147,0.107391,0.316871
min,1.510000,0.000000,0.000000,1.520000,0.000000e+00,-0.801946,1.530000,1.560000,0.697647,1.400000e+03,3.650000e+03,0.052553,1.520000,0.010000,0.054966,0.083336,0.195840,0.000000,0.000000,0.000000,0.000000,0.000820,0.000156,0.007701,0.004727,0.204893,0.024735,2.278172,1.004784,0.000000,0.000000,5.955932,3.856003,0.000000,1.590000,0.000000,1.660000,0.000000,0.000000,2.338095,9.537198,0.058734,-2596.260000,0.000000,0.000000,0.000000,0.000000,0.004000,0.002000,0.000000e+00,0.016760,0.000000,1.648352,0.000000,1.534733,1.570486,-549.955166,1.560000,0.017674,1.621932,-535.784277,-0.792602,-0.386182
25%,18.200000,18.500000,17.832500,18.185000,9.640650e+04,-0.010883,18.212500,18.244333,0.977496,1.107546e+05,1.200750e+05,0.776877,18.172500,0.570000,0.643670,0.668347,0.895714,18.500000,17.822500,0.000000,0.000000,0.118567,0.108471,0.136552,0.131005,12.998161,10.876837,17.122294,14.932155,18.000000,10.000000,28.734533,17.376758,17.380000,19.072500,16.550000,19.810000,25.692814,28.373682,34.389471,41.443836,0.756791,-0.850000,0.000000,0.000000,0.098000,0.098000,0.139333,0.132000,4.420353e-01,0.690424,30.653572,40.843242,0.657237,18.188817,18.392234,-2.990191,18.244333,0.532968,19.725504,16.767035,-0.038692,0.280051
50%,83.900000,85.000000,81.465000,83.830000,3.052910e+05,0.000000,83.461000,82.598667,0.996471,3.343506e+05,3.5380

In [ ]:
all_data['ATR_5P']=all_data['ATR_5']/all_data['Close']*100
all_data['ATR_15P']=all_data['ATR_15']/all_data['Close']*100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
all_data.head()

,Symbol,Date,Open,High,Low,Close,Volume,Return,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio,prev_close,TR,ATR_5,ATR_15,ATR_Ratio,prev_high,prev_low,plus DM,minus DM,plus DM_5,minus DM_5,plus DM_15,minus DM_15,plus DI_5,minus DI_5,plus DI_15,minus DI_15,DX_5,DX_15,ADX_5,ADX_15,Lowest_5D,High_5D,Lowest_15D,High_15D,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,Diff,Up,Down,avg_5up,avg_5down,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD,15MA,SD,upperband,lowerband,RC,Bollinger,ATR_5P,ATR_15P
29,SRF,1999-02-15 15:29:59.712,1.80,1.98,1.80,1.95,6800.0,0.083333,2.044,2.234000,1.092955,22990.0,11590.000000,1.983607,1.80,0.18,0.282575,0.252131,1.120748,2.10,1.77,0.00,0.00,0.032178,0.084839,0.058952,0.063305,11.387366,30.023454,23.381448,25.107880,45.0,4.0,40.548306,19.066667,1.77,2.39,1.77,3.12,29.032258,13.333333,14.963859,37.089506,0.403453,0.15,0.15,0.00,0.060,0.140,0.073333,0.076667,0.428571,0.956522,30.000000,48.888889,0.613636,2.030716,2.177635,0.146919,2.234000,0.276245,2.786490,1.681510,-0.025000,0.242982,14.491029,12.929780
30,SRF,1999-02-16 15:29:59.712,1.83,1.90,1.58,1.60,81100.0,-0.179487,1.924,2.203333,1.145184,38650.0,16856.666667,2.292861,1.95,0.37,0.300060,0.259989,1.154127,1.98,1.80,0.00,0.22,0.025742,0.111871,0.055022,0.073751,8.579045,37.282883,21.163111,28.367018,63.0,15.0,45.038645,18.795556,1.58,2.39,1.58,3.12,2.469136,1.298701,15.457686,33.991704,0.454749,-0.35,0.00,0.35,0.060,0.180,0.069333,0.100000,0.333333,0.693333,25.000000,40.944882,0.610577,1.887144,2.105430,0.218286,2.203333,0.319143,2.841620,1.565047,-0.223301,0.027380,18.753753,16.249292
31,SRF,1999-02-17 15:29:59.712,1.83,1.97,1.65,1.94,29500.0,0.212500,1.842,2.192667,1.190373,44150.0,18573.333333,2.377064,1.60,0.37,0.314048,0.267323,1.174790,1.90,1.58,0.07,0.00,0.034594,0.089497,0.056020,0.068834,11.015458,28.497815,20.956033,25.749509,44.0,10.0,44.830916,18.209185,1.58,2.39,1.58,3.12,44.444444,23.376623,17.679908,32.089802,0.550951,0.34,0.34,0.00,0.098,0.180,0.089333,0.100000,0.544444,0.893333,35.251799,47.183099,0.747128,1.904763,2.084752,0.179989,2.192667,0.325455,2.843577,1.541757,-0.076190,0.305913,16.188043,13.779524
32,SRF,1999-02-18 15:29:59.712,1.94,1.99,1.50,1.71,59300.0,-0.118557,1.800,2.170667,1.205926,45240.0,22430.000000,2.016942,1.94,0.49,0.349238,0.282168,1.237697,1.97,1.65,0.00,0.15,0.027675,0.101597,0.052286,0.074245,7.924404,29.091146,18.529946,26.312470,57.0,17.0,47.264733,18.128573,1.50,2.10,1.50,3.12,35.000000,12.962963,22.929908,30.639351,0.748381,-0.23,0.00,0.23,0.098,0.140,0.089333,0.111333,0.700000,0.802395,41.176471,44.518272,0.924934,1.839842,2.037908,0.198066,2.170667,0.346955,2.864578,1.476756,-0.161765,0.168065,20.423300,16.501047
33,SRF,1999-02-19 15:29:59.712,1.83,1.90,1.60,1.80,80850.0,0.052632,1.800,2.157333,1.198519,51510.0,27620.000000,1.864953,1.71,0.30,0.339391,0.283357,1.197751,1.99,1.50,0.00,0.00,0.022140,0.081278,0.048800,0.069296,6.523469,23.948199,17.222058,24.455272,57.0,17.0,49.211786,18.053335,1.50,1.99,1.50,3.12,61.224490,18.518519,34.434066,33.851419,1.017212,0.09,0.09,0.00,0.116,0.116,0.095333,0.108667,1.000000,0.877301,50.000000,46.732026,1.069930,1.826561,2.008169,0.181608,2.157333,0.357660,2.872654,1.442013,-0.100000,0.250229,18.855041,15.742040


In [ ]:
all_data.describe()

,Open,High,Low,Close,Volume,Return,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio,prev_close,TR,ATR_5,ATR_15,ATR_Ratio,prev_high,prev_low,plus DM,minus DM,plus DM_5,minus DM_5,plus DM_15,minus DM_15,plus DI_5,minus DI_5,plus DI_15,minus DI_15,DX_5,DX_15,ADX_5,ADX_15,Lowest_5D,High_5D,Lowest_15D,High_15D,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,Diff,Up,Down,avg_5up,avg_5down,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD,15MA,SD,upperband,lowerband,RC,Bollinger,ATR_5P,ATR_15P
count,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000
mean,850.173593,860.338065,838.954900,849.496132,9.989041e+05,0.001251,848.424588,845.646043,0.997781,9.982127e+05,9.975058e+05,1.003990,848.977886,22.584624,22.499188,22.332951,0.993399,859.798581,838.439363,5.225818,4.884045,5.216038,4.850803,5.175737,4.816019,22.429155,19.984040,22.786596,20.532666,40.633399,25.088900,40.641643,25.110702,822.075140,874.502594,797.509768,892.638980,51.004175,53.218730,51.007978,51.009553,1.003095,0.518246,5.837438,5.319192,5.835016,5.290702,5.810301,5.259532,inf,1.510437,51.882350,52.619702,1.002663,848.414628,845.655537,-2.759091,845.646043,22.565460,890.776963,800.515123,0.016320,0.537381,3.900034,3.889203
std,1812.590602,1831.394861,1791.165492,1810.209439,2.410711e+06,0.039498,1808.664057,1804.772787,0.040329,2.200063e+06,2.125259e+06,0.352309,1809.662649,58.447335,47.939390,45.475700,0.157639,1830.793660,1790.617342,30.237842,30.164457,13.753803,14.373002,11.056135,11.503320,12.767377,12.133773,8.439246,8.287301,25.850335,17.748107,15.537752,10.307473,1760.790514,1858.615729,1716.786605,1893.622569,28.524019,28.776343,20.959008,13.389602,0.355480,43.230312,30.566040,29.537656,17.272888,16.986920,13.514274,13.667435,NaN,1.543278,27.490902,16.667758,0.602604,1808.500314,1804.416229,39.116880,1804.772787,54.488836,1893.009702,1718.917147,0.107391,0.316871,2.717071,2.109302
min,1.510000,0.000000,0.000000,1.520000,0.000000e+00,-0.801946,1.530000,1.560000,0.697647,1.400000e+03,3.650000e+03,0.052553,1.520000,0.010000,0.054966,0.083336,0.195840,0.000000,0.000000,0.000000,0.000000,0.000820,0.000156,0.007701,0.004727,0.204893,0.024735,2.278172,1.004784,0.000000,0.000000,5.955932,3.856003,0.000000,1.590000,0.000000,1.660000,0.000000,0.000000,2.338095,9.537198,0.058734,-2596.260000,0.000000,0.000000,0.000000,0.000000,0.004000,0.002000,0.000000e+00,0.016760,0.000000,1.648352,0.000000,1.534733,1.570486,-549.955166,1.560000,0.017674,1.621932,-535.784277,-0.792602,-0.386182,0.891526,1.117645
25%,18.200000,18.500000,17.832500,18.185000,9.640650e+04,-0.010883,18.212500,18.244333,0.977496,1.107546e+05,1.200750e+05,0.776877,18.172500,0.570000,0.643670,0.668347,0.895714,18.500000,17.822500,0.000000,0.000000,0.118567,0.108471,0.136552,0.131005,12.998161,10.876837,17.122294,14.932155,18.000000,10.000000,28.734533,17.376758,17.380000,19.072500,16.550000,19.810000,25.692814,28.373682,34.389471,41.443836,0.756791,-0.850000,0.000000,0.000000,0.098000,0.098000,0.139333,0.132000,4.420353e-01,0.690424,30.653572,40.843242,0.657237,18.188817,18.392234,-2.990191,18.244333,0.532968,19.725504,16.767035,-0.038692,0.280051,2.453288,2.599209
5

In [ ]:
all_data['MACDP'] = (all_data['5Ewm'] - all_data['15Ewm'])/ all_data['15Ewm']*100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
all_data.describe()

,Open,High,Low,Close,Volume,Return,SMA_5,SMA_15,SMA_ratio,SMA5_Volume,SMA15_Volume,SMA_Volume_Ratio,prev_close,TR,ATR_5,ATR_15,ATR_Ratio,prev_high,prev_low,plus DM,minus DM,plus DM_5,minus DM_5,plus DM_15,minus DM_15,plus DI_5,minus DI_5,plus DI_15,minus DI_15,DX_5,DX_15,ADX_5,ADX_15,Lowest_5D,High_5D,Lowest_15D,High_15D,Stochastic_5,Stochastic_15,Stochastic_%D_5,Stochastic_%D_15,Stochastic_Ratio,Diff,Up,Down,avg_5up,avg_5down,avg_15up,avg_15down,RS_5,RS_15,RSI_5,RSI_15,RSI_ratio,5Ewm,15Ewm,MACD,15MA,SD,upperband,lowerband,RC,Bollinger,ATR_5P,ATR_15P,MACDP
count,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,2.027000e+04,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000,20270.000000
mean,850.173593,860.338065,838.954900,849.496132,9.989041e+05,0.001251,848.424588,845.646043,0.997781,9.982127e+05,9.975058e+05,1.003990,848.977886,22.584624,22.499188,22.332951,0.993399,859.798581,838.439363,5.225818,4.884045,5.216038,4.850803,5.175737,4.816019,22.429155,19.984040,22.786596,20.532666,40.633399,25.088900,40.641643,25.110702,822.075140,874.502594,797.509768,892.638980,51.004175,53.218730,51.007978,51.009553,1.003095,0.518246,5.837438,5.319192,5.835016,5.290702,5.810301,5.259532,inf,1.510437,51.882350,52.619702,1.002663,848.414628,845.655537,-2.759091,845.646043,22.565460,890.776963,800.515123,0.016320,0.537381,3.900034,3.889203,0.319177
std,1812.590602,1831.394861,1791.165492,1810.209439,2.410711e+06,0.039498,1808.664057,1804.772787,0.040329,2.200063e+06,2.125259e+06,0.352309,1809.662649,58.447335,47.939390,45.475700,0.157639,1830.793660,1790.617342,30.237842,30.164457,13.753803,14.373002,11.056135,11.503320,12.767377,12.133773,8.439246,8.287301,25.850335,17.748107,15.537752,10.307473,1760.790514,1858.615729,1716.786605,1893.622569,28.524019,28.776343,20.959008,13.389602,0.355480,43.230312,30.566040,29.537656,17.272888,16.986920,13.514274,13.667435,NaN,1.543278,27.490902,16.667758,0.602604,1808.500314,1804.416229,39.116880,1804.772787,54.488836,1893.009702,1718.917147,0.107391,0.316871,2.717071,2.109302,2.984450
min,1.510000,0.000000,0.000000,1.520000,0.000000e+00,-0.801946,1.530000,1.560000,0.697647,1.400000e+03,3.650000e+03,0.052553,1.520000,0.010000,0.054966,0.083336,0.195840,0.000000,0.000000,0.000000,0.000000,0.000820,0.000156,0.007701,0.004727,0.204893,0.024735,2.278172,1.004784,0.000000,0.000000,5.955932,3.856003,0.000000,1.590000,0.000000,1.660000,0.000000,0.000000,2.338095,9.537198,0.058734,-2596.260000,0.000000,0.000000,0.000000,0.000000,0.004000,0.002000,0.000000e+00,0.016760,0.000000,1.648352,0.000000,1.534733,1.570486,-549.955166,1.560000,0.017674,1.621932,-535.784277,-0.792602,-0.386182,0.891526,1.117645,-23.877880
25%,18.200000,18.500000,17.832500,18.185000,9.640650e+04,-0.010883,18.212500,18.244333,0.977496,1.107546e+05,1.200750e+05,0.776877,18.172500,0.570000,0.643670,0.668347,0.895714,18.500000,17.822500,0.000000,0.000000,0.118567,0.108471,0.136552,0.131005,12.998161,10.876837,17.122294,14.932155,18.000000,10.000000,28.734533,17.376758,17.380000,19.072500,16.550000,19.810000,25.692814,28.373682,34.389471,41.443836,0.756791,-0.850000,0.000000,0.000000,0.098000,0.098000,0.139333,0.132000,4.420353e-01,0.690424,30.653572,40.843242,0.657237,18.188817,18.392234,-2.990191,18.244333,0.532968,19.725504,

In [ ]:
all_data.to_csv('alldata.csv')